In [167]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cftime

import sys
sys.path.append('home/links/ct715/eddy_feedback/')
import functions.eddy_feedback as ef
import functions.data_wrangling as data

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in divide")

### Maths Servers
This script is for use on the Maths department machines.

In [168]:
path = '/home/links/ct715/data_storage/PAMIP/monthly/1.1_pdSST-pdSIC/'

# CESM1-WACCM-SC

In [169]:
# cesm1_ua = xr.open_mfdataset(
#     path + 'ua/CESM1-WACCM-SC/U_redate_zonmean_1700-2000_sub_1.1.nc',
#     parallel=True,
#     cache=False
# )
# cesm1_epfy = xr.open_mfdataset(
#     path + 'epfy/CESM1-WACCM-SC/EPY_redate_mon_PAMIP-1.1-QBO_300m_sub.nc',
#     parallel=True,
#     cache=False
# )

# # manipulate dataArrays and save new dataset
# cesm1_ua = data.data_checker1000(cesm1_ua)
# cesm1_epfy = data.data_checker1000(cesm1_epfy)
# cesm1_ua.attrs, cesm1_epfy.attrs = {}, {}
# cesm1_epfy, cesm1_ua = cesm1_epfy.isel(lon=0), cesm1_ua.isel(lon=0)
# cesm1_ds = xr.Dataset( {'ubar': cesm1_ua.U, 'epfy': cesm1_epfy.EPY})
# cesm1_ds = ef.calculate_divFphi(cesm1_ds)
# cesm1_ds

In [170]:
# import processed dataset:
cesm1_ds = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/CESM1-WACCM-SC_1.1_ubar_epfy_divFy.nc',
    parallel=True,
    chunks={'time':30}
)

# new method
cesm1_EFP = ef.calculate_efp_pamip(cesm1_ds, which_div1='divFy')

# old method
waccm = cesm1_ds.where(cesm1_ds.level < 1000.)
waccm = waccm.where(waccm.level > 1.)
waccm = waccm.where(waccm.lat > -90.)
waccm = waccm.where(waccm.lat < 90)
waccm = waccm.interp(lat=np.arange(-90,93,3))
waccm = ef.calculate_divFphi(waccm, save_divFphi='divF')
waccm_EFP = ef.calculate_efp_pamip(waccm, which_div1='divF')

print(f'New EFP: {cesm1_EFP}\nOld EFP: {waccm_EFP}')

New EFP: 0.1325
Old EFP: 0.1446


In [171]:
# cesm1_ds.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/CESM1-WACCM-SC_1.1_ubar_epfy_divFy.nc')

# CNRM-CM6-1

In [172]:
# cnrm_ua = xr.open_mfdataset(
#     path + 'ua/CNRM-CM6-1/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )
# cnrm_epfy = xr.open_mfdataset(
#     path + 'epfy/CNRM-CM6-1/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )

# # optional changes
# # cnrm_ua.attrs, cnrm_epfy.attrs = {}, {}
# # cnrm_ua = cnrm_ua.isel(lon=0)
# cnrm_epfy = cnrm_epfy.isel(lon=0)
# cnrm_epfy, cnrm_ua = cnrm_epfy.sel(time=slice('2000-06', '2001-05')), cnrm_ua.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# cnrm_ds = xr.Dataset( {'ubar': cnrm_ua.ua.mean('lon'), 'epfy': cnrm_epfy.epfy})
# cnrm_ds = data.data_checker1000(cnrm_ds)
# cnrm_ds = ef.calculate_divFphi(cnrm_ds)
# cnrm_ds

In [173]:
# import processed dataset:
cnrm_ds = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/CNRM-CM6-1_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
cnrm_EFP = ef.calculate_efp_pamip(cnrm_ds, which_div1='divFy')

# old method
cnrm_cm6 = cnrm_ds.where(cnrm_ds.level < 1000.)
cnrm_cm6 = cnrm_cm6.where(cnrm_cm6.level > 1.)
cnrm_cm6 = cnrm_cm6.where(cnrm_cm6.lat > -90.)
cnrm_cm6 = cnrm_cm6.where(cnrm_cm6.lat < 90)
cnrm_cm6 = cnrm_cm6.interp(lat=np.arange(-90,93,3))
cnrm_cm6 = ef.calculate_divFphi(cnrm_cm6, save_divFphi='divF')
cnrm_cm6_EFP = ef.calculate_efp_pamip(cnrm_cm6, which_div1='divF')

print(f'New EFP: {cnrm_EFP}\nOld EFP: {cnrm_cm6_EFP}')

New EFP: 0.2469
Old EFP: 0.2819


In [174]:
# cnrm_ds.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/CNRM-CM6-1_1.1_ubar_epfy_divFy.nc')

# E3SMv1

In [175]:
# e3sm_ua = xr.open_mfdataset(
#     path + 'ua/E3SMv1/*.nc',
#     parallel=True,
#     cache=False
# )
# e3sm_epfy = xr.open_mfdataset(
#     path + 'epfy/E3SMv1/*.nc',
#     parallel=True,
#     cache=False
# )

# # optional changes
# e3sm_ua.attrs, e3sm_epfy.attrs = {}, {}
# # e3sm_ua = e3sm_ua.isel(lon=0)
# e3sm_epfy = e3sm_epfy.isel(lon=0)

# hadll_ua, hadll_epfy = data.change_to_cftime(hadll_ua), data.change_to_cftime(hadll_epfy)
# # e3sm_epfy, e3sm_ua = e3sm_epfy.sel(time=slice('2000-06', '2001-05')), e3sm_ua.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# e3sm_ds = xr.Dataset( {'ubar': e3sm_ua.U.mean('lon'), 'epfy': e3sm_epfy.EPYpe})
# e3sm_ds = ef.calculate_divFphi(e3sm_ds)
# e3sm_ds

In [176]:
# import processed dataset:
e3sm_ds = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/E3SMv1_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
e3sm_EFP = ef.calculate_efp_pamip(e3sm_ds, which_div1='divFy')

# old method
e3smv1 = e3sm_ds.where(e3sm_ds.level < 1000.)
e3smv1 = e3smv1.where(e3smv1.level > 1.)
e3smv1 = e3smv1.where(e3smv1.lat > -90.)
e3smv1 = e3smv1.where(e3smv1.lat < 90)
e3smv1 = e3smv1.interp(lat=np.arange(-90,93,3))
e3smv1 = ef.calculate_divFphi(e3smv1, save_divFphi='divF')
e3smv1_EFP = ef.calculate_efp_pamip(e3smv1, which_div1='divF')

print(f'New EFP: {e3sm_EFP}\nOld EFP: {e3smv1_EFP}')

New EFP: 0.0533
Old EFP: 0.0584


In [177]:
# e3sm_ds.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/E3SMv1_1.1_ubar_epfy_divFy.nc')

# EC-EARTH3

In [178]:
# ec_earth_ua = xr.open_mfdataset(
#     path + 'ua/EC-EARTH3/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )
# ec_earth_epfy = xr.open_mfdataset(
#     path + 'epfy/EC-EARTH3/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )

# # optional changes
# ec_earth_ua = data.data_checker1000(ec_earth_ua, check_vars=False)
# ec_earth_epfy = data.data_checker1000(ec_earth_epfy, check_vars=False)
# # ec_earth_ua.attrs, ec_earth_epfy.attrs = {}, {}
# # ec_earth_ua = ec_earth_ua.isel(lon=0)
# ec_earth_epfy = ec_earth_epfy.mean('lon')
# # ec_earth_epfy, ec_earth_ua = ec_earth_epfy.sel(time=slice('2000-06', '2001-05')), ec_earth_ua.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# ec_earth_ds = xr.Dataset( {'ubar': ec_earth_ua.ua.mean('lon'), 'epfy': ec_earth_epfy.epfy})
# ec_earth_ds = ef.calculate_divFphi(ec_earth_ds)
# ec_earth_ds

In [179]:
# import processed dataset:
ec_earth_ds = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/EC-EARTH3_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
ec_earth_EFP = ef.calculate_efp_pamip(ec_earth_ds, which_div1='divFy')

# old method
earth3 = ec_earth_ds.where(ec_earth_ds.level < 1000.)
earth3 = earth3.where(earth3.level > 1.)
earth3 = earth3.where(earth3.lat > -90.)
earth3 = earth3.where(earth3.lat < 90)
earth3 = earth3.interp(lat=np.arange(-90,93,3))
earth3 = ef.calculate_divFphi(earth3, save_divFphi='divF')
earth3_EFP = ef.calculate_efp_pamip(earth3, which_div1='divF')

print(f'New EFP: {ec_earth_EFP}\nOld EFP: {earth3_EFP}')

New EFP: 0.3003
Old EFP: 0.3538


In [180]:
# ec_earth_ds.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/EC-EARTH3_1.1_ubar_epfy_divFy.nc')

# ECHAM6.3_AWI

In [181]:
# echam_ua = xr.open_mfdataset(
#     path + 'ua/ECHAM6.3_AWI/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )
# echam_epfy = xr.open_mfdataset(
#     path + 'epfy/ECHAM6.3_AWI/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )

# # optional changes
# echam_ua = data.data_checker1000(echam_ua, check_vars=False)
# echam_epfy = data.data_checker1000(echam_epfy, check_vars=False)
# # echam_ua.attrs, echam_epfy.attrs = {}, {}
# echam_epfy = echam_epfy.isel(lon=0)
# # echam_epfy = echam_epfy.mean('lon')

# echam_epfy = echam_epfy.sel( level = echam_ua.level.values)

# echam_ua, echam_epfy = data.change_to_cftime(echam_ua), data.change_to_cftime(echam_epfy)
# echam_epfy, echam_ua = echam_epfy.sel(time=slice('2000-06', '2001-05')), echam_ua.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# echam_ds = xr.Dataset( {'ubar': echam_ua.ua.mean('lon'), 'epfy': echam_epfy.epfy})
# echam_ds = ef.calculate_divFphi(echam_ds)
# echam_ds

In [182]:
# import processed dataset:
echam_ds = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/ECHAM6.3_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
echam_EFP = ef.calculate_efp_pamip(echam_ds, which_div1='divFy')

# old method
awi = echam_ds.where(echam_ds.level < 1000.)
awi = awi.where(awi.level > 1.)
awi = awi.where(awi.lat > -90.)
awi = awi.where(awi.lat < 90)
awi = awi.interp(lat=np.arange(-90,93,3))
awi = ef.calculate_divFphi(awi, save_divFphi='divF')
awi_EFP = ef.calculate_efp_pamip(awi, which_div1='divF')

print(f'New EFP: {echam_EFP}\nOld EFP: {awi_EFP}')

New EFP: 0.0134
Old EFP: 0.0142


In [183]:
# echam_ds.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/ECHAM6.3_1.1_ubar_epfy_divFy.nc')

# HadGEM3-GC31-LL

In [184]:
# hadll_ua = xr.open_mfdataset(
#     path + 'ua/HadGEM3-GC31-LL/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )
# hadll_epfy = xr.open_mfdataset(
#     path + 'epfy/HadGEM3-GC31-LL/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )

# # rename ua components
# hadll_ua['ua'] = hadll_ua['u']
# hadll_ua = hadll_ua.ua.rename(
#     {'p_1': 'level', 'latitude_1': 'lat', 'longitude_1': 'lon', 't': 'time'}
# )

# # optional changes
# hadll_ua = data.data_checker1000(hadll_ua, check_vars=False)
# hadll_epfy = data.data_checker1000(hadll_epfy, check_vars=False)

# # hadll_ua.attrs, hadll_epfy.attrs = {}, {}
# # hadll_epfy = hadll_epfy.isel(lon=0)
# # hadll_epfy = hadll_epfy.mean('lon')
# # hadll_epfy = hadll_epfy.sel( level = hadll_ua.level.values)

# # hadll_ua, hadll_epfy = data.change_to_cftime(hadll_ua), data.change_to_cftime(hadll_epfy)
# hadll_epfy, hadll_ua = hadll_epfy.sel(time=slice('2000-06', '2001-05')), hadll_ua.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# hadll_ds = xr.Dataset( {'ubar': hadll_ua.mean('lon'), 'epfy': hadll_epfy.epfy})
# hadll_ds = ef.calculate_divFphi(hadll_ds)
# hadll_ds

In [185]:
# import processed dataset:
hadll_ds = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/HadGEM3-GC31-LL_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
hadll_EFP = ef.calculate_efp_pamip(hadll_ds, which_div1='divFy')

# old method
gc31_ll = hadll_ds.where(hadll_ds.level < 1000.)
gc31_ll = gc31_ll.where(gc31_ll.level > 1.)
gc31_ll = gc31_ll.where(gc31_ll.lat > -90.)
gc31_ll = gc31_ll.where(gc31_ll.lat < 90)
gc31_ll = gc31_ll.interp(lat=np.arange(-90,93,3))
gc31_ll = ef.calculate_divFphi(gc31_ll, save_divFphi='divF')
gc31_ll_EFP = ef.calculate_efp_pamip(gc31_ll, which_div1='divF')

print(f'New EFP: {hadll_EFP}\nOld EFP: {gc31_ll_EFP}')

New EFP: 0.278
Old EFP: 0.3216


In [186]:
# hadll_ds.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/HadGEM3-GC31-LL_1.1_ubar_epfy_divFy.nc')

# IPSL-CM6A-LR

In [187]:
# ipsl_ua = xr.open_mfdataset(
#     path + 'ua/IPSL-CM6A-LR/*.nc',
#     parallel=True,
#     cache=False
# )
# ipsl_epfy = xr.open_mfdataset(
#     path + 'epfy/IPSL-CM6A-LR/*.nc',
#     parallel=True,
#     cache=False
# )

# # model specific changes:
# ipsl_epfy['time'] = ipsl_ua.time
# ipsl_epfy.attrs = {}
# ipsl_epfy = ipsl_epfy.rename({'record': 'ens_ax'})
# ipsl_ua = ipsl_ua.rename({'plev': 'level', 'record': 'ens_ax'})
# ipsl_ua['level'] = ipsl_ua.level / 100

# # match pressure levels to smaller dataset
# if len(ipsl_epfy.level) > len(ipsl_ua.level):
#     ipsl_epfy = ipsl_epfy.sel( level = ipsl_ua.level.values )
# else:
#     ipsl_ua = ipsl_ua.sel( level = ipsl_epfy.level.values )

# # optional changes
# ipsl_ua.attrs, ipsl_epfy.attrs = {}, {}
# # ipsl_ua = ipsl_ua.isel(lon=0)
# # ipsl_epfy = ipsl_epfy.isel(lon=0)

# # hadll_ua, hadll_epfy = data.change_to_cftime(hadll_ua), data.change_to_cftime(hadll_epfy)
# # ipsl_epfy, ipsl_ua = ipsl_epfy.sel(time=slice('2000-06', '2001-05')), ipsl_ua.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# ipsl_ds = xr.Dataset( {'ubar': ipsl_ua.ua.mean('lon'), 'epfy': ipsl_epfy.epfy})
# ipsl_ds = ipsl_ds.sel(time=slice('2000-06', '2001-05'))
# ipsl_ds = ef.calculate_divFphi(ipsl_ds)
# ipsl_ds

In [188]:
# import processed dataset:
ipsl_ds = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/IPSL-CM6A-LR_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
ipsl_EFP = ef.calculate_efp_pamip(ipsl_ds, which_div1='divFy')

# old method
ipsl_cm6a = ipsl_ds.where(ipsl_ds.level < 1000.)
ipsl_cm6a = ipsl_cm6a.where(ipsl_cm6a.level > 1.)
ipsl_cm6a = ipsl_cm6a.where(ipsl_cm6a.lat > -90.)
ipsl_cm6a = ipsl_cm6a.where(ipsl_cm6a.lat < 90)
ipsl_cm6a = ipsl_cm6a.interp(lat=np.arange(-90,93,3))
ipsl_cm6a = ef.calculate_divFphi(ipsl_cm6a, save_divFphi='divF')
ipsl_cm6a_EFP = ef.calculate_efp_pamip(ipsl_cm6a, which_div1='divF')

print(f'New EFP: {ipsl_EFP}\nOld EFP: {ipsl_cm6a_EFP}')

New EFP: 0.2762
Old EFP: 0.309


In [189]:
# ipsl_ds.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/IPSL-CM6A-LR_1.1_ubar_epfy_divFy.nc')

# OpenIFS-159

In [190]:
# # open159_ua = xr.open_mfdataset(
# #     path + 'ua/OpenIFS-159/*.nc',
# #     combine='nested',
# #     concat_dim='ens_ax',
# #     parallel=True,
# #     cache=False
# # )
# open159 = xr.open_mfdataset(
#     path + 'epfy/OpenIFS-159/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )

# # optional changes
# open159 = data.data_checker1000(open159, check_vars=False)
# # open159.attrs, open159.attrs = {}, {}
# open159 = open159.isel(lon=0)
# # open159 = open159.mean('lon')
# # open159 = open159.sel( level = open159.level.values)
# # open159, open159 = data.change_to_cftime(open159), data.change_to_cftime(open159)
# # open159, open159 = open159.sel(time=slice('2000-06', '2001-05')), open159.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# open159 = xr.Dataset( {'ubar': open159.ua, 'epfy': open159.epfy})
# open159 = ef.calculate_divFphi(open159)
# open159

In [191]:
# import processed dataset:
open159 = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/OpenIFS-159_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
open159_EFP = ef.calculate_efp_pamip(open159, which_div1='divFy')

# old method
ifs159 = open159.where(open159.level < 1000.)
ifs159 = ifs159.where(ifs159.level > 1.)
ifs159 = ifs159.where(ifs159.lat > -90.)
ifs159 = ifs159.where(ifs159.lat < 90)
ifs159 = ifs159.interp(lat=np.arange(-90,93,3))
ifs159 = ef.calculate_divFphi(ifs159, save_divFphi='divF')
ifs159_EFP = ef.calculate_efp_pamip(ifs159, which_div1='divF')

print(f'New EFP: {open159_EFP}\nOld EFP: {ifs159_EFP}')

New EFP: 0.3328
Old EFP: 0.3825


In [192]:
# open159.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/OpenIFS-159_1.1_ubar_epfy_divFy.nc')

# OpenIFS-511

In [193]:
# # open511_ua = xr.open_mfdataset(
# #     path + 'ua/OpenIFS-159/*.nc',
# #     combine='nested',
# #     concat_dim='ens_ax',
# #     parallel=True,
# #     cache=False
# # )
# open511 = xr.open_mfdataset(
#     path + 'epfy/OpenIFS-511/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )

# # optional changes
# open511 = data.data_checker1000(open511, check_vars=False)
# # open511.attrs, open511.attrs = {}, {}
# open511 = open511.isel(lon=0)
# open511 = open511.sel(time=slice('2000-06', '2001-05'))

# # make dataset and calculate divFy
# open511 = xr.Dataset( {'ubar': open511.ua, 'epfy': open511.epfy})
# open511 = ef.calculate_divFphi(open511)
# open511

In [194]:
# import processed dataset:
open511 = xr.open_mfdataset(
    '/home/links/ct715/data_storage/PAMIP/processed_monthly/OpenIFS-511_1.1_ubar_epfy_divFy.nc',
    parallel=True,
)

# new method
open511_EFP = ef.calculate_efp_pamip(open511, which_div1='divFy')

# old method
ifs511 = open511.where(open511.level < 1000.)
ifs511 = ifs511.where(ifs511.level > 1.)
ifs511 = ifs511.where(ifs511.lat > -90.)
ifs511 = ifs511.where(ifs511.lat < 90)
ifs511 = ifs511.interp(lat=np.arange(-90,93,3))
ifs511 = ef.calculate_divFphi(ifs511, save_divFphi='divF')
ifs511_EFP = ef.calculate_efp_pamip(ifs511, which_div1='divF')

print(f'New EFP: {open511_EFP}\nOld EFP: {ifs511_EFP}')

New EFP: 0.2325
Old EFP: 0.2751


In [195]:
# open511.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/OpenIFS-511_1.1_ubar_epfy_divFy.nc')

# OpenIFS-1279

In [196]:
# open1279_ua = xr.open_mfdataset(
#     path + 'ua/OpenIFS-159/*.nc',
#     combine='nested',
#     concat_dim='ens_ax',
#     parallel=True,
#     cache=False
# )
open1279 = xr.open_mfdataset(
    path + 'epfy/OpenIFS-1279/*.nc',
    combine='nested',
    concat_dim='ens_ax',
    parallel=True,
    cache=False
)
open1279


<xarray.Dataset>
Dimensions:     (time: 2, ens_ax: 20, bnds: 2, lon: 1, lat: 480, plev: 19)
Coordinates:
  * time        (time) datetime64[ns] 2000-04-16 2000-05-16T09:00:00
  * lon         (lon) float64 0.0
  * lat         (lat) float64 -89.81 -89.44 -89.06 -88.69 ... 89.06 89.44 89.81
  * plev        (plev) float64 1e+05 9.25e+04 8.5e+04 ... 1e+03 500.0 100.0
Dimensions without coordinates: ens_ax, bnds
Data variables: (12/15)
    time_bnds   (ens_ax, time, bnds) datetime64[ns] dask.array<chunksize=(1, 2, 2), meta=np.ndarray>
    va          (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    wa          (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    utendepfdy  (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    utendepfdz  (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    ta          (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    ...          ...
    psitem      (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    epfy        (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    epfz        (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    utendvtem   (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    utendwtem   (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
    utendepfd   (ens_ax, time, plev, lat, lon) float64 dask.array<chunksize=(1, 2, 19, 480, 1), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.5 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Mon Jan 18 16:32:51 2021: cdo monmean /p/largedata/hhb19/js...
    frequency:    mon
    CDO:          Climate Data Operators version 1.9.5 (http://mpimet.mpg.de/...

In [197]:
# open511.to_netcdf('/home/links/ct715/data_storage/PAMIP/processed_monthly/OpenIFS-511_1.1_ubar_epfy_divFy.nc')